In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from collections import Counter
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import re
import unicodedata
import string
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import euclidean_distances

In [3]:
# import spacy.cli
# spacy.cli.download("en_core_web_lg")

In [4]:
!pip install -U -q PyDrive

In [5]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [6]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [7]:
#https://drive.google.com/file/d/1IF0zUGOa-gpsftrs7-wLWkaELkSW7oAQ/view?usp=sharing
#https://drive.google.com/file/d/1IF0zUGOa-gpsftrs7-wLWkaELkSW7oAQ/view?usp=sharing
fileDownloaded = drive.CreateFile({'id':'1IF0zUGOa-gpsftrs7-wLWkaELkSW7oAQ'})
fileDownloaded.GetContentFile('Arabic_names.csv')

In [8]:
letters = ['ل','م','ن','ه','و','ي','ء','ا','آ','أ','ب','ة','ت','ث','ج','ح','خ','د','ذ','ر','ز','س','ش','ص','ض','ط','ظ','ع','غ','ف','ق','ك']
len(letters)

32

In [119]:
data_females = pd.read_csv("/content/females_ar.csv",header=0)
data_females

,Name,Gender
0,آداب,Female
1,آسية,Female
2,آلاء,Female
3,آمال,Female
4,الآيات,Female
...,...,...
1660,يُسر,Female
1661,يُسراء,Female
1662,يقظانة,Female
1663,يمامة,Female


In [120]:
data_males = pd.read_csv("/content/males_ar.csv",header=0)
data_males

,Name,Gender
0,آدم,Male
1,آسر,Male
2,أبد,Male
3,أبدع,Male
4,أبره,Male
...,...,...
1418,يانع,Male
1419,يحيى,Male
1420,يقظان,Male
1421,يمان,Male


In [121]:
m_names = np.array(data_males["Name"])
m_names

array(['آدم', 'آسر', 'أبد', ..., 'يقظان', 'يمان', 'ينيع'], dtype=object)

In [122]:
f_names = np.array(data_females["Name"])
f_names

array(['آداب', 'آسية', 'آلاء', ..., 'يقظانة', 'يمامة', 'يُمنى'],
      dtype=object)

In [123]:
print("Number of Male names : ",len(m_names))
print("Number of Female names : ",len(f_names))

Number of Male names :  1423
Number of Female names :  1665


# **Creating Real Names**

In [124]:
RealFemales = []
RealMales = []

In [125]:
for f_n in f_names:
  RealFemales.append(f_n + " " + m_names[np.random.randint(0,len(m_names))]+ " " + m_names[np.random.randint(0,len(m_names))])

RealFemales

['آداب فضل جندب',
 'آسية داهش باسل',
 'آلاء ضاوي لاحظ',
 'آمال سُرَّي نفوذ',
 'الآيات جبل حرملة',
 'إباء عديل نافل',
 'إبتكار رهيم سفدان',
 'إبتهال رجب رائق',
 'إتفاق عبدالوهّاب مبروك',
 'إجلال عرادة كرم',
 'إحتفاظ دعاس روحان',
 'إخلاص قرير عبدالتواب',
 'إدراك ذائد سفيد',
 'أذينة عبدالبارئ مبروك',
 'أرب لامة فدائي',
 'أرزة أسهم ابره',
 'ارتياد قرير درع',
 'أرواح عتبة شاهي',
 'أزاهير زهير عرابة',
 'أزهار إلياس نابغ',
 'أسرار غانم مرداس',
 'أسما خضر ناصح',
 'أسوة خليل حارث',
 'أشواق أمير رضى',
 'اصطلاح اوس عزام',
 'أصيلة بشار ثري',
 'أضواء شريح خازن',
 'اعتماد بّراق عبدالعظيم',
 'إعجاز ضرار عبدالمجيد',
 'أعناب راكن بشارة',
 'إغراء عبدالمجيد قليب',
 'إفتخار حاكم زنيم',
 'أفراح افضل إلفي',
 'أفكار مادن صافي',
 'إقبال حاسم صابر',
 'إكتمال زكوان ذهني',
 'ألى صباحي سفيان',
 'ألغاز بشير عاشور',
 'أماثل عبدالجبار زكنون',
 'إمامة فهد سلام',
 'أماني عائد اسر',
 'إمتلاك اصلان دامس',
 'أمل عامر راهف',
 'أمنية أرجح زهدي',
 'أميرة ثائر حارث',
 'أمنية عابدين سكرية',
 'انتصار ريان صراح',
 'انسجام كايد 

In [126]:
for m_n in m_names:
  RealMales.append(m_n + " " + m_names[np.random.randint(0,len(m_names))]+ " " + m_names[np.random.randint(0,len(m_names))])
  
RealMales

['آدم ركين عرندس',
 'آسر مخزوم راكز',
 'أبد زكوان رامق',
 'أبدع غراس صائم',
 'أبره عيشي داني',
 'أبسل رتيب كاظم',
 'أبسم لاهز قارس',
 'أبشر صادح سباق',
 'أبلج لقيط ظالم',
 'أبي سائب مبتغي',
 'أجاد أرشد غرام',
 'أجزل لهيم عرفان',
 'أجمل دحلان ناهض',
 'أجوب راني نعيم',
 'أجود وردان جبري',
 'أحسب سجيل ارجح',
 'أحسن أيوب قادر',
 'أحشم مجاهد أشرف',
 'أحمد زكنون وادع',
 'أحيد رزان حبيش',
 'أخزم نور دعاس',
 'أخير زعزع أفضل',
 'أخيل صخر أصلان',
 'أُدد عدي كحيل',
 'أدرد غياث عاد',
 'أدرم أبشر رقيم',
 'أدغم شهيد شهوان',
 'أدلم باقي اديب',
 'أدهم أفصح وجيه',
 'أديب عامر كاتب',
 'أربد إسماعيل راتق',
 'أرجح دارع أرجح',
 'أرجوان أريج شهاب',
 'أرحب رمضان وردان',
 'أرحم اسامة رابح',
 'أرسلان حسام خزاعي',
 'أرشد شماخ قاصد',
 'أرغد سديف غنيم',
 'أرقم شكري عابر',
 'أركان متمم عبدالتواب',
 'أريج رئال زينالعابدين',
 'أزهد محمود حماد',
 'أزهر سفيد زنيم',
 'أسامة لميع صارم',
 'أسد عبدالحق كريم',
 'أسعد جامع ضياء',
 'أسلع غلاب دراك',
 'أسلم أكرم هيصم',
 'أسمح غيهب اكثم',
 'أسهم سعيد شاهب',
 'أسيد غضنفر لهيف',

In [127]:
print("Number of Male names : ",len(RealMales))
print("Number of Female names : ",len(RealFemales))
print("Sum Of Names : ",len(RealMales)+len(RealFemales))

Number of Male names :  1423
Number of Female names :  1665
Sum Of Names :  3088


In [128]:
Real_Names = RealMales + RealFemales
Real_Names[1422:1426]

['ينيع ضياف عبدالوالي', 'آداب فضل جندب', 'آسية داهش باسل', 'آلاء ضاوي لاحظ']

# **Define ways of faking names.**

In [129]:
def type_1_fake (s):
  s = s.replace(s[np.random.randint(0,len(s))],"")
  return s

def type_2_fake (s):
  s = s.replace(s[np.random.randint(0,len(s))],letters[np.random.randint(0,32)])
  return s

def type_3_fake (s):
  s = s.replace(s[np.random.randint(0,len(s))],letters[np.random.randint(0,32)]+letters[np.random.randint(0,32)])
  return s

In [130]:
# Testing our metods of faking names
ss = "اسامة"
l = type_1_fake(ss)
ll = type_2_fake(ss)
lll = type_3_fake(ss)
print("type_1_fake : ",l)
print("type_2_fake : ",ll)
print("type_3_fake : ",lll)

type_1_fake :  اساة
type_2_fake :  اصامة
type_3_fake :  قثسقثمة


# **Generating Fake Names**

In [131]:
# Change in all three words
Fake_Names = []
type_two = []
for idx in range(len(Real_Names)):
  splited_name = Real_Names[idx].split(" ")
  print("\n------REAL------\n",splited_name)
  Num = np.random.randint(1,4)
  if Num == 1 :
    for i in range(len(splited_name)):
      splited_name[i] = type_1_fake(splited_name[i])
    print("\n------FAKE------\n",splited_name)
    Fake_Names.append(splited_name[0]+" "+splited_name[1]+" "+splited_name[2])
    
  if Num == 2 :
    for j in range(len(splited_name)):
      splited_name[j] = type_2_fake(splited_name[j])
    print("\n------FAKE------\n",splited_name)
    Fake_Names.append(splited_name[0]+" "+splited_name[1]+" "+splited_name[2])
    type_two.append(idx)

  if Num == 3 :
    for n in range(len(splited_name)):
      splited_name[n] = type_3_fake(splited_name[n])
    print("\n------FAKE------\n",splited_name)
    Fake_Names.append(splited_name[0]+" "+splited_name[1]+" "+splited_name[2])

  

Streaming output truncated to the last 5000 lines.
------FAKE------
 ['سَوسح', 'عبدالمجند', 'رازد']

------REAL------
 ['سيسبانة', 'سفيد', 'صنوان']

------FAKE------
 ['سيسباة', 'سفد', 'صنان']

------REAL------
 ['سيماء', 'غراس', 'نزيه']

------FAKE------
 ['سيماند', 'ظضراس', 'نسليه']

------REAL------
 ['شادية', 'عنتر', 'طارق']

------FAKE------
 ['شادطة', 'عنتغ', 'طاضق']

------REAL------
 ['شارحة', 'عرندس', 'بدر']

------FAKE------
 ['شارحي', 'عرنذس', 'برر']

------REAL------
 ['شافية', 'بلسم', 'اسحاق']

------FAKE------
 ['شافزة', 'بآسم', 'اسءاق']

------REAL------
 ['شامخة', 'شداد', 'عميم']

------FAKE------
 ['شاموةة', 'شدنغد', 'ضجميم']

------REAL------
 ['شاهدة', 'دليل', 'دريد']

------FAKE------
 ['شاصدة', 'دظيظ', 'ررير']

------REAL------
 ['شاهقة', 'عكنان', 'راضي']

------FAKE------
 ['شاقة', 'عكا', 'اضي']

------REAL------
 ['شبك', 'دامث', 'سائر']

------FAKE------
 ['شلك', 'رامث', 'سائغ']

------REAL------
 ['شدا', 'أحيد', 'كحيل']

------FAKE------
 ['شا', 'حيد', 'كحي']

-

In [134]:
# Change in randomly selected number of words 

Fake_Names1 = []
for idx in range(len(Real_Names)):
  splited_name = Real_Names[idx].split(" ")
  print("\n------REAL------\n",splited_name)
  Num = np.random.randint(1,4)
  if Num == 1 :
    Num_of_words_1 = np.random.randint(1,4)
    for i in range(Num_of_words_1):
      splited_name[i] = type_1_fake(splited_name[i])
    print("\n------FAKE------\n",splited_name)
    Fake_Names1.append(splited_name[0]+" "+splited_name[1]+" "+splited_name[2])
    
  if Num == 2 :
    Num_of_words_2 = np.random.randint(1,4)
    for j in range(Num_of_words_2):
      splited_name[j] = type_2_fake(splited_name[j])
    print("\n------FAKE------\n",splited_name)
    Fake_Names1.append(splited_name[0]+" "+splited_name[1]+" "+splited_name[2])

  if Num == 3 :
    Num_of_words_3 = np.random.randint(1,4)
    for n in range(Num_of_words_3):
      splited_name[n] = type_3_fake(splited_name[n])
    print("\n------FAKE------\n",splited_name)
    Fake_Names1.append(splited_name[0]+" "+splited_name[1]+" "+splited_name[2])

  

Streaming output truncated to the last 5000 lines.
------FAKE------
 ['هدَوهدن', 'عبدالمجيد', 'رائد']

------REAL------
 ['سيسبانة', 'سفيد', 'صنوان']

------FAKE------
 ['آليآلبانة', 'سفيد', 'صنوان']

------REAL------
 ['سيماء', 'غراس', 'نزيه']

------FAKE------
 ['سيمصء', 'غراس', 'نزيه']

------REAL------
 ['شادية', 'عنتر', 'طارق']

------FAKE------
 ['شازية', 'عنتر', 'طارق']

------REAL------
 ['شارحة', 'عرندس', 'بدر']

------FAKE------
 ['شاحة', 'رندس', 'بدر']

------REAL------
 ['شافية', 'بلسم', 'اسحاق']

------FAKE------
 ['ثافية', 'بلسس', 'اسحاق']

------REAL------
 ['شامخة', 'شداد', 'عميم']

------FAKE------
 ['شامخه', 'شنان', 'مميم']

------REAL------
 ['شاهدة', 'دليل', 'دريد']

------FAKE------
 ['شهدة', 'دي', 'دريد']

------REAL------
 ['شاهقة', 'عكنان', 'راضي']

------FAKE------
 ['ضساهقة', 'عكنتون', 'راضي']

------REAL------
 ['شبك', 'دامث', 'سائر']

------FAKE------
 ['شفنك', 'دامث', 'سائر']

------REAL------
 ['شدا', 'أحيد', 'كحيل']

------FAKE------
 ['شدح', 'أحيد', 'كلي

In [135]:
print(len(Fake_Names))
print(len(Fake_Names1))
print(len(Real_Names))

3088
3088
3088


In [136]:
all_names = Real_Names + Fake_Names
all_names1 = Real_Names + Fake_Names1
print(len(all_names))
print(len(all_names1))

6176
6176


In [137]:
vectorizer = CountVectorizer(lowercase=False)
features = vectorizer.fit_transform(all_names).todense()
features

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [138]:
from sklearn.metrics.pairwise import cosine_similarity
distances = []
s = []
for i in range(3088):
  if cosine_similarity(features[i],features[i+3088]) > 0.4 :
    distances.append(1)
    s.append(all_names[i+3088])
    s.append(all_names[i])
  else:
    distances.append(0)
    


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.w

In [139]:
pd.Series(distances).value_counts()

0    3084
1       4
dtype: int64

In [140]:
s

['إحتفاظ دتاس روحان',
 'إحتفاظ دعاس روحان',
 'لشولق عارف سراج',
 'اشواق عارف سراج',
 'ُبى ريم بدالوهّاب',
 'رُبى كريم عبدالوهّاب',
 'ُلفى َدّاد فيق',
 'زُلفى سَدّاد شفيق']

In [141]:
features_1 = vectorizer.fit_transform(all_names1).todense()
features_1

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [142]:
distances_1 = []
s_1 = []
for i in range(3088):
  if cosine_similarity(features_1[i],features_1[i+3088]) > 0.9 :
    distances_1.append(1)
    s_1.append(all_names1[i+3088])
    s_1.append(all_names1[i])

  else:
    distances_1.append(0)
    


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.w

In [143]:
pd.Series(distances_1).value_counts()

0    3079
1       9
dtype: int64

In [144]:
s_1

['جمان داغر أفضل',
 'جمان داغر أفضل',
 'ُسن سيف خالص',
 'حُسن سيف خالص',
 'شاهرة صارم عبدالماجد',
 'شاهرة صارم عبدالماجد',
 'َرف سابق شهدي',
 'شَرف سابق شهدي',
 'ظهور فهمي اويس',
 'ظهور فهمي اويس',
 'عطيَّ أنف فائز',
 'عطيَّة أنف فائز',
 'ِسمة حام بشاش',
 'قِسمة حام بشاش',
 'خُليكة صابر رفقي',
 'مُليكة صابر رفقي',
 'َنال عارم أفخم',
 'مَنال عارم أفخم']

### So we need Real Name to balance the data  
- Number of Male names :  1423
- Number of Female names :  1665

In [145]:
RealFemales_new = []
for i in range(1874):
  if i > 1664:
    RealFemales_new.append(f_names[i-1664] + " " + m_names[np.random.randint(0,len(m_names))]+ " " + m_names[np.random.randint(0,len(m_names))])
  else:
    RealFemales_new.append(f_names[i] + " " + m_names[np.random.randint(0,len(m_names))]+ " " + m_names[np.random.randint(0,len(m_names))])
len(RealFemales_new)

1874

In [146]:
RealMales_new = []
for i in range(1249):
  if i > 551:
    RealMales_new.append(m_names[i-1422] + " " + m_names[np.random.randint(0,len(m_names))]+ " " + m_names[np.random.randint(0,len(m_names))])
  else:
    RealMales_new.append(m_names[i] + " " + m_names[np.random.randint(0,len(m_names))]+ " " + m_names[np.random.randint(0,len(m_names))])
len(RealMales_new)

1249

In [147]:
RealNames_new = RealFemales_new + RealMales_new
len(RealNames_new)

3123

In [148]:
print("Total Real Names Number (0) : ",len(Real_Names) + len(RealNames_new))
print("Total Fake Names Number (1) : ",2801)


Total Real Names Number (0) :  6211
Total Fake Names Number (1) :  2801


### Total Real Names Number (1) 
### Total Fake Names Number (0)

In [149]:
Total_Fake = []
Total_Real = []

In [150]:
len(distances)

3088

In [151]:
distances[0:10]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [152]:
len(all_names)

6176

In [153]:
all_names[3088]

'آدذ رضين عرندس'

In [154]:
all_names1[3088]

'آم ركي عرنس'

In [155]:
Total_Fake = []
Total_Real = []

for i in range(3088):
  if distances[i] == 0 :
    Total_Fake.append(all_names[i+3088])
  else : 
    Total_Real.append(all_names[i+3088])
    
for i in range(3088):
  if distances_1[i] == 0 :
    Total_Fake.append(all_names1[i+3088])
  else : 
    Total_Real.append(all_names1[i+3088])


Total_Real = Total_Real + Real_Names + RealNames_new
print(f"Checking the total numbers of R & F : \nTotal Real Names Number (0) : {len(Total_Real)}\nTotal Fake Names Number (1) : {len(Total_Fake)}")

Checking the total numbers of R & F : 
Total Real Names Number (0) : 6224
Total Fake Names Number (1) : 6163


In [156]:
data_list = []
label_list = []

for i in range(len (Total_Fake)):
  data_list.append(Total_Real[i])
  label_list.append(1)
  data_list.append(Total_Fake[i])
  label_list.append(0)

print(len(data_list))
print(len(label_list))

12326
12326


In [157]:
Total_Real

['إحتفاظ دتاس روحان',
 'لشولق عارف سراج',
 'ُبى ريم بدالوهّاب',
 'ُلفى َدّاد فيق',
 'جمان داغر أفضل',
 'ُسن سيف خالص',
 'شاهرة صارم عبدالماجد',
 'َرف سابق شهدي',
 'ظهور فهمي اويس',
 'عطيَّ أنف فائز',
 'ِسمة حام بشاش',
 'خُليكة صابر رفقي',
 'َنال عارم أفخم',
 'آدم ركين عرندس',
 'آسر مخزوم راكز',
 'أبد زكوان رامق',
 'أبدع غراس صائم',
 'أبره عيشي داني',
 'أبسل رتيب كاظم',
 'أبسم لاهز قارس',
 'أبشر صادح سباق',
 'أبلج لقيط ظالم',
 'أبي سائب مبتغي',
 'أجاد أرشد غرام',
 'أجزل لهيم عرفان',
 'أجمل دحلان ناهض',
 'أجوب راني نعيم',
 'أجود وردان جبري',
 'أحسب سجيل ارجح',
 'أحسن أيوب قادر',
 'أحشم مجاهد أشرف',
 'أحمد زكنون وادع',
 'أحيد رزان حبيش',
 'أخزم نور دعاس',
 'أخير زعزع أفضل',
 'أخيل صخر أصلان',
 'أُدد عدي كحيل',
 'أدرد غياث عاد',
 'أدرم أبشر رقيم',
 'أدغم شهيد شهوان',
 'أدلم باقي اديب',
 'أدهم أفصح وجيه',
 'أديب عامر كاتب',
 'أربد إسماعيل راتق',
 'أرجح دارع أرجح',
 'أرجوان أريج شهاب',
 'أرحب رمضان وردان',
 'أرحم اسامة رابح',
 'أرسلان حسام خزاعي',
 'أرشد شماخ قاصد',
 'أرغد سديف غنيم',
 'أرقم

In [158]:
Total_Fake

['آدذ رضين عرندس',
 'آر خزو اكز',
 'أد كوان راق',
 'أكتدع غرشآس روائم',
 'أبه عش داي',
 'أبست رويب كاظق',
 'أبس اهز ارس',
 'ظتبشر حهادح شبباق',
 'أبلعت لقيآق ظالخء',
 'أبد قائب مبشغي',
 'أساد أطشد غصام',
 'أجز ليم عفان',
 'أجمصآ دحلادا ناهلن',
 'أجددب رانخو نشزيم',
 'أجكقد لفردان جبرأي',
 'أسب جيل اجح',
 'جحسن أيثب قاور',
 'أطشم مجاهد أترف',
 'آثحمد ذصكنون زيادع',
 'أحيب دزان حبيب',
 'أخزد غور دةاس',
 'أير عع أفض',
 'أسيل يخر أصلآن',
 'أجذدد عةكي كحسسل',
 'أقرق غياث ظاد',
 'أدمم أصشر رقجم',
 'أغم شهي شهان',
 'أفالم باقوأ اءنيب',
 'أده فصح جيه',
 'أدب عام كتب',
 'أرد إسماعي راق',
 'أجح دار أرح',
 'أطثجوان أرءطج شهاظح',
 'أرح رضان وران',
 'ورحم اسامذ راأح',
 'أرسلا حسم خاعي',
 'جثرشد شماوآ قاصار',
 'أرخد طديف غنيك',
 'أعآقم رجكري عابغث',
 'أرظان يتيي عبدذلتوذب',
 'أيج رئا ينالعابدين',
 'زهد حود حمد',
 'أمظهر هعفيد زنيدت',
 'أساختة بهميع صقفرم',
 'ألد عبعالحق كأيم',
 'أسع جام ضيا',
 'أهلع غلاة دراس',
 'أسددم جءكرم هيصظو',
 'أمقمح غيهصن ةقكثم',
 'لخسهم فشعيد دأاهب',
 'أسصغد ارضنفر لهيفش',


In [159]:
data_dic_1 = { "Name": data_list,
             "Label": label_list }
data_1 = pd.DataFrame(data_dic_1)
data_1

,Name,Label
0,إحتفاظ دتاس روحان,1
1,آدذ رضين عرندس,0
2,لشولق عارف سراج,1
3,آر خزو اكز,0
4,ُبى ريم بدالوهّاب,1
...,...,...
12321,يقظان نبل زكوان,0
12322,قادر نامق قاسط,1
12323,ياة فيل عدي,0
12324,قادس شافعي دهمان,1


In [115]:
data_dic_2 = { "Name": data_list,
             "Label": label_list }
data_2 = pd.DataFrame(data_dic_2)
data_2

,Name,Label
0,اماثل عبدالماجد دؤضب,1
1,آثكم نجضةبالدضةن كساطس,0
2,اهيم شركة امجد,1
3,آسهر بهائحف نوراذحذحه,0
4,حائم عارد رافة,1
...,...,...
12301,يقظاة نشي اهر,0
12302,فرحون اليف ضرغام,1
12303,يوخاوخة ضاردط رؤوحخ,0
12304,فريد عبدالحي نظام,1


In [160]:
data_final = pd.concat([data_1,data_2],axis=0)

In [161]:
data_final

,Name,Label
0,إحتفاظ دتاس روحان,1
1,آدذ رضين عرندس,0
2,لشولق عارف سراج,1
3,آر خزو اكز,0
4,ُبى ريم بدالوهّاب,1
...,...,...
12301,يقظاة نشي اهر,0
12302,فرحون اليف ضرغام,1
12303,يوخاوخة ضاردط رؤوحخ,0
12304,فريد عبدالحي نظام,1


In [162]:
data_final.to_csv("Names_data_all.csv",index=False)